# 🏆 Submission

## Load libraries, models, metadata and clusting information

In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras

model1 = keras.models.load_model('../input/modelv1-cluster1of2/Model')
model2 = keras.models.load_model('../input/modelv1-cluster2of2/Model')

df_clusters = pd.read_parquet('../input/2-clusters/2_cluster.parquet')
df_clusters = df_clusters.set_index('investment_id').T.to_dict('list')


metadata = pd.read_parquet('../input/metadata/metadata')

2022-03-05 10:20:05.605217: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


## Load competition enviroment

In [2]:
import ubiquant
env = ubiquant.make_env()  
iter_test = env.iter_test()

## Loop over test data and provide preditions
Models selected based on which cluster it belongs to.

In [3]:
for (test_df, sample_prediction_df) in iter_test:

    for i in range(300):
        test_df[['f_'+str(i)]] = test_df[['f_'+str(i)]].subtract(metadata['mean'][i])
        test_df[['f_'+str(i)]] = test_df[['f_'+str(i)]]/(metadata['std'][i] + 1e-12)
    
    test_df[['time_id','inv_id']] = test_df.row_id.str.split("_",expand=True)
    test_df = test_df.set_index(['investment_id','time_id'])
    test_df = test_df.drop(['row_id','inv_id'],axis = 1)
    
    inv_id = test_df.index.levels[0]
    
    test = [test_df.loc[investment].values for investment in inv_id]
    test = np.array(test, dtype=np.float32)
    
    
    pred1 = model1.predict(test)
    pred2 = model2.predict(test)
    
#     pred2 = np.reshape(pred2,len(test_df))
#     sample_prediction_df['target'] = pred2

    inv_count = 0
    for investment in test_df.index.levels[0]:

        time_ids = test_df.loc[investment].index

        for index in range(test.shape[1]):  
            row_id = time_ids[index]+'_'+str(investment)

            for j in sample_prediction_df.index[sample_prediction_df['row_id']==row_id]:
                
                if df_clusters[int(investment)][0]==1:
                    sample_prediction_df.loc[j,('target')] = pred1[inv_count][index][0].astype(float)
                else:
                    sample_prediction_df.loc[j,('target')] = pred2[inv_count][index][0].astype(float)

        inv_count=+1
        
    env.predict(sample_prediction_df)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


2022-03-05 10:20:09.731028: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
